# Sample script for Hybrid VAE model

### imports

In [1]:
%matplotlib inline

from torch.autograd import Variable
from torch import optim
from torch.utils.data import DataLoader
import torch
import matplotlib.pyplot as plt
import numpy as np

import sys
sys.path.append("../../Modules")
sys.path.append("../../Datasets")

# local imports
from visualize import printText
from models import AdvancedHybridVAE
from babelDatasets.sentiment140 import Sentiment140
from babelDatasets.utils import padding_merge

## Define dataset loader

In [2]:
batch_size = 32

dataset = Sentiment140(data_directory="../../Datasets/Data",train=False,max_sentence_size=32)
data_loader = DataLoader(dataset,batch_size=batch_size,shuffle=True,collate_fn=padding_merge)

num_classes = 82

batch_loader = iter(data_loader)
# input has shape [batch_size,seq_len,1]

# N sents: 52990  train: False  sentences_path: ../../Datasets/Data/test_sentences.txt


## Define and load model

In [4]:
model = AdvancedHybridVAE(input_size=1,conv_size=256,rnn_size=256,latent_size=64,output_size=num_classes,use_softmax=True)
model.eval()
model.load_state_dict(torch.load("../../Saved_models/Sentiment140/HybridVAE_nll_map.pt"))

## Collect some samples and plot

In [9]:
def sampleCompare(batch_loader,model,data_decoder):
    sampled_data = {"x":[],"recon_x":[]}
    try:
        batch = next(batch_loader)[0]
    except StopIteration:
        batch_loader = iter(data_loader)
        batch = next(batch_loader)[0]
    x = Variable(torch.FloatTensor(batch)).unsqueeze(2).transpose(1,0)
    size = (x.size()[0],x.size()[1],model.samplelayer.outputShape()[-1])
    z = 2*Variable(torch.FloatTensor(*size).normal_(),requires_grad=False)
    recon_x = model.sample(z,x.size()[0])
    _,topi = recon_x.data.topk(1)
    pred_x = Variable(topi)
    sampled_data["x"] = x.squeeze(2)
    sampled_data["recon_x"] = pred_x.squeeze(2)
    printText(sampled_data,data_decoder,recon_x_text="Sample")

In [10]:
sampleCompare(batch_loader,model,dataset.encoderDecoder)


True:
 @user thanks! just asked.******
Sample:
 I r Shop********** yeuon. u lrd

True:
 playing guitar*****************
Sample:
 @use n'*  lad**oo*s*me wer'liy*

True:
 What a night*******************
Sample:
 ihd SOY****** @o at an..oritng 

True:
 @user i did that last night****
Sample:
 mo iver*i*********ae d. He wach

True:
 the sims 3*********************
Sample:
 I es uo ge u  oooieep  !.rea  y

True:
 @user  vote for me? ;)*********
Sample:
 I aAcan.s..*ah Hahe?*****do  he

True:
 @user coffee is on the way*****
Sample:
 ieady TA  ww.po* deet. dor oo a

True:
 @user thanks sweetie***********
Sample:
 ie a  *e Su.yeey?..***haterish 

True:
 Teen Choice Awards*************
Sample:
 iweyrmacl tto m*****n.***** oo 

True:
 I just managed it!*************
Sample:
 ioa i weliss sy m me fkinl.*** 

True:
 says good evening!  @url*******
Sample:
 Ioneed.**..*****goa go*try anee
